James Caldwell <br>
9/11/25 <br>


In [2]:
import pandas as pd
import numpy as np

# snapshots = pd.read_csv('')

In [2]:
snapshots_backup = snapshots.copy()

In [29]:
print(list(snapshots.columns))


['Student_System_ID', 'Term', 'Dataset', 'Academic_Career', '%StudentCareerKey', 'Class_Num', 'Class_Dim_ID', 'Academic_Career_Desc', 'Academic_Program', 'Academic_Program_Desc', 'Earn_Credit', 'Enrollment_Add_Date', 'Enrollment_Drop_Date', 'Enrollment_Status_Reason', 'Grade_Category', 'Grade_Points', 'Grading_Basis_Audit', 'Grading_Basis', 'Official_Grade', 'Repeat_Code', 'Repeat_Code_Desc', 'Repeat_Date', 'Session_Code', 'Session_Desc', 'Student_Enrollment_Status', 'Units_Attempted', 'Units_Earned', 'Units_Taken', 'Units_Taken_Academic_Progress', 'Instructor_ID', 'Instructor_Name', 'Course_ID', 'Class_Academic_Career', 'Class_Academic_Career_Desc', 'Class_Academic_Group', 'Class_Academic_Group_Desc', 'Class_Academic_Org', 'Class_Academic_Org_Desc', 'Assoc_Class', 'Catalog_Number', 'Class_Section', 'Comb_Sections_Desc', 'Class_Title', 'Class_Type', 'Class_Component', 'Course_Contact_Hours', 'Course_Count', 'Class_Dt_End', 'Class_Dt_Start', 'Enrollment_Capacity', 'Enrollment_Total', 'I

In [14]:
enrollment_cols = [col for col in snapshots.columns if "enrollment" in col.lower()]
enrollment_cols


['Enrollment_Add_Date',
 'Enrollment_Drop_Date',
 'Enrollment_Status_Reason',
 'Student_Enrollment_Status',
 'Enrollment_Capacity',
 'Enrollment_Total',
 'Min_Enrollment_Num']

In [58]:
# Uncomment to use backup
snapshots = snapshots_backup.copy()
snapshots["Audit_Only_Flag"] .value_counts()

Audit_Only_Flag
N    398492
Y        81
Name: count, dtype: int64

Enrolled for credit
No Audit only 

In [1]:

# ----------------------------
# Step 1. Apply population filters
# ----------------------------
snapshots = snapshots[
    (
        (snapshots["Units_Taken_for_Progress"] > 0) |
        (snapshots["Primary_Academic_Program"].isin(["MDG", "MDGN"])) |
        (snapshots["Form_of_Study"].isin([
            "International Study - FT", # 7696 records
            "International Study - PT", # 0
            "Domestic Study - FT", # 0
            "Domestic Study - PT", # 0
            "Exchange - Domestic", # 0
            "Exchange - International" # 488 records
            # Affiliated Study 535?
            # Detached Study 3?
        ]))
    )
    & (snapshots["Withdraw_Cancel"].isin(["---", "Withdrew"]))
]

# ----------------------------
# Step 2. Collapse academic career
# ----------------------------
snapshots.loc[snapshots["Academic_Career"] != "UGRD", "Academic_Career"] = "GRAD"

# ----------------------------
# Step 3. Assign major info (first non-null plan/major)
# ----------------------------
def pick_first_major(row):
    prefixes = [
        ("MAJ", 1),
        ("CRT", 1),
        ("COS", 1),
        ("SP", 1),
        # PRP not in dataset, skip
        ("MIN", 1),
        ("MJ2", 1)
    ]
    for prefix, idx in prefixes:
        desc_col = f"{prefix}_Major_Minor_Desc_{idx}"
        deglev_col = f"{prefix}_Degree_Level_{idx}"
        degree_col = f"{prefix}_Degree_{idx}"
        cip_col = f"{prefix}_SCHEV_CIP_{idx}"
        if pd.notna(row.get(desc_col, None)) and str(row[desc_col]).strip() != "":
            return row[desc_col], row[deglev_col], row[degree_col], row[cip_col]
    return None, None, None, None

snapshots[["majdesc", "majdeglev", "majdegree", "majschevcip"]] = snapshots.apply(
    pick_first_major, axis=1, result_type="expand"
)

# ----------------------------
# Step 4. Degree-seeking classification
# ----------------------------
# deg_seeking = ["BACH", "DOCT", "EDS", "JD", "MAST", "MD","CERT"]
deg_seeking = ['BA', 'BACH', 'CE', 'CERT', 'CO', 'COS', 'DO', 'DOCT', 'ED', 'EDS', 'JD', 'MA', 'MAST', 'MD']
non_deg = ["PND", "SP", "COS", "PRP"]

snapshots["degreeseekingstatus"] = snapshots["majdeglev"].map(
    lambda x: "Degree Seeking" if x in deg_seeking else
              "Non Degree Seeking" if x in non_deg else None
)

# Special override for CERT with CIP ≠ 92.0005 or 92.0006
mask = (snapshots["majdeglev"] == "CERT") & (~snapshots["majschevcip"].isin(["92.0005", "92.0006"]))
snapshots.loc[mask, "degreeseekingstatus"] = "Degree Seeking"

# ----------------------------
# Step 5. Keep only degree-seeking 
# ----------------------------
snapshots = snapshots[snapshots["degreeseekingstatus"] == "Degree Seeking"]

# Step 5b exclude auditors
snapshots = snapshots[snapshots["Audit_Only_Flag"] == "N"]

# ----------------------------
# Step 6. Deduplicate to first record per student per career
# ----------------------------
snapshots = snapshots.sort_values(["Student_System_ID", "Academic_Career"])
snapshots = snapshots.drop_duplicates(subset=["Student_System_ID", "Academic_Career"], keep="first")

# This did nothing
HS_students = pd.read_excel(r"V:\FISAP\hs_students_2425.xlsx")
snapshots = snapshots[~snapshots["Student_System_ID"].isin(HS_students["Student_System_ID"])]
# ----------------------------
# Step 7. Summary counts
# ----------------------------
summary = snapshots["Academic_Career"].value_counts().reset_index()
summary.columns = ["Academic_Career", "Number"]

print("University of Virginia")
print("FISAP - Unduplicated Headcount, 2024-2025")
print(summary)


NameError: name 'snapshots' is not defined